# Funcionamiento del parametro inference_batch_size

## torchmdnet/data.py

### Configuración del Entorno y Importación de Bibliotecas

Para comenzar, importamos las bibliotecas necesarias para nuestro módulo, que incluyen:

- `os.path.join`: Utilizado para unir diferentes partes de una ruta de archivo.
- `tqdm`: Proporciona una barra de progreso visual para iteraciones.
- `torch`: La biblioteca principal de PyTorch.
- `torch.utils.data.Subset`: Permite crear un subconjunto de un conjunto de datos.
- `torch_geometric.loader.DataLoader`: DataLoader específico para datos en formato geométrico en PyTorch.
- `pytorch_lightning.LightningDataModule`: Clase base para definir los módulos de datos en PyTorch Lightning.
- `pytorch_lightning.utilities.rank_zero_warn`: Utilizado para mostrar advertencias solo en el proceso de rango cero.
- `torchmdnet.datasets`: Importa los conjuntos de datos necesarios para nuestro módulo.
- `torch_geometric.data.Dataset`: Clase base para definir conjuntos de datos en formato geométrico en PyTorch.
- `torchmdnet.utils.make_splits`: Función para dividir un conjunto de datos en conjuntos de entrenamiento, validación y prueba.
- `torchmdnet.utils.MissingEnergyException`: Excepción personalizada para manejar casos en los que falte energía en el conjunto de datos.
- `torch_scatter.scatter`: Función para realizar operaciones de reducción (scatter) en datos dispersos.

Además, importamos `dtype_mapping` de `torchmdnet.models.utils`, que probablemente sea un mapeo de tipos de datos utilizado en los modelos.


In [ ]:
from os.path import join
from tqdm import tqdm
import torch
from torch.utils.data import Subset
from torch_geometric.loader import DataLoader
from pytorch_lightning import LightningDataModule
from pytorch_lightning.utilities import rank_zero_warn
from torchmdnet import datasets
from torch_geometric.data import Dataset
from torchmdnet.utils import make_splits, MissingEnergyException
from torch_scatter import scatter
from torchmdnet.models.utils import dtype_mapping

### Clase `FloatCastDatasetWrapper`

Esta clase es una envoltura (wrapper) alrededor de un conjunto de datos existente y tiene la intención de convertir los datos a un tipo de dato de punto flotante específico. Aquí está una descripción de sus componentes:

- `FloatCastDatasetWrapper(Dataset)`: Esta clase hereda de `torch_geometric.data.Dataset`, lo que significa que actúa como un tipo de conjunto de datos en formato geométrico en PyTorch.

- `__init__(self, dataset, dtype=torch.float64)`: El método de inicialización de la clase toma dos argumentos: `dataset`, que es el conjunto de datos que se envuelve, y `dtype`, que es el tipo de dato al que se deben convertir los datos. Por defecto, el tipo de dato es `torch.float64`.

- `super(FloatCastDatasetWrapper, self).__init__(dataset.root, dataset.transform, dataset.pre_transform, dataset.pre_filter)`: Llama al constructor de la clase base `Dataset` con los mismos argumentos que el conjunto de datos original, es decir, `root`, `transform`, `pre_transform`, y `pre_filter`.

- `self.dataset = dataset`: Almacena una referencia al conjunto de datos original.

- `self.dtype = dtype`: Almacena el tipo de dato al que se deben convertir los datos.

Este wrapper permite mantener la funcionalidad del conjunto de datos original mientras se realiza una conversión de tipo de dato a flotante. Esto puede ser útil para asegurar la consistencia en el tipo de dato de entrada para modelos de aprendizaje automático.


In [ ]:
class FloatCastDatasetWrapper(Dataset):
    def __init__(self, dataset, dtype=torch.float64):
        super(FloatCastDatasetWrapper, self).__init__(
            dataset.root, dataset.transform, dataset.pre_transform, dataset.pre_filter
        )
        self.dataset = dataset
        self.dtype = dtype

### Método `len`

Este método sobrecarga la función `len` para devolver la longitud del conjunto de datos envuelto por la instancia de `FloatCastDatasetWrapper`. Aquí está una descripción de su funcionalidad:

- `def len(self)`: Este método define una función llamada `len` que toma `self` como argumento, lo que significa que opera en una instancia de la clase `FloatCastDatasetWrapper`.

- `return len(self.dataset)`: Dentro del método, se llama a la función `len` en el conjunto de datos original (`self.dataset`) y se devuelve su longitud. Esto permite que la instancia de `FloatCastDatasetWrapper` se comporte como un conjunto de datos estándar en términos de su longitud, proporcionando una interfaz consistente para su uso en iteraciones y otras operaciones que dependen de conocer la longitud del conjunto de datos.


In [ ]:
    def len(self):
        return len(self.dataset)

### Método `get`

Este método se utiliza para obtener un dato del conjunto de datos envuelto por la instancia de `FloatCastDatasetWrapper` y convertir los tensores a un tipo de dato de punto flotante específico. Aquí está una descripción de su funcionalidad:

- `def get(self, idx)`: Define un método llamado `get` que toma `idx` como argumento, indicando el índice del dato que se quiere obtener del conjunto de datos envuelto.

- `data = self.dataset.get(idx)`: Se llama al método `get` en el conjunto de datos original (`self.dataset`) para obtener el dato correspondiente al índice `idx`.

- `for key, value in data`: Se itera sobre los pares clave-valor en el dato obtenido.

- `if torch.is_tensor(value) and torch.is_floating_point(value)`: Se comprueba si el valor es un tensor de punto flotante.

- `setattr(data, key, value.to(self.dtype))`: Si el valor es un tensor de punto flotante, se convierte a `self.dtype` (el tipo de dato especificado) y se establece como atributo en el dato.

- `return data`: Se devuelve el dato modificado, donde los tensores han sido convertidos al tipo de dato especificado.


In [ ]:
    def get(self, idx):
        data = self.dataset.get(idx)
        for key, value in data:
            if torch.is_tensor(value) and torch.is_floating_point(value):
                setattr(data, key, value.to(self.dtype))
        return data

### Método `__getattr__`

Este método se utiliza para interceptar y manejar la obtención de atributos que no están definidos explícitamente en la clase `FloatCastDatasetWrapper`. Aquí está una descripción de su funcionalidad:

- `def __getattr__(self, name)`: Define un método especial llamado `__getattr__` que se activa cuando se intenta acceder a un atributo que no está definido explícitamente en la clase.

- `if hasattr(self.dataset, name)`: Verifica si el atributo existe en el conjunto de datos subyacente (`self.dataset`).

- `return getattr(self.dataset, name)`: Si el atributo existe en el conjunto de datos subyacente, devuelve el valor del atributo llamando a `getattr` en el conjunto de datos.

- `raise AttributeError(...)`: Si el atributo no existe ni en la clase `FloatCastDatasetWrapper` ni en su conjunto de datos subyacente, se genera una excepción `AttributeError` que indica que el atributo no se puede encontrar.

Este método permite que la clase `FloatCastDatasetWrapper` delegue la obtención de atributos a su conjunto de datos subyacente, lo que garantiza que pueda acceder a todos los atributos y métodos del conjunto de datos original sin tener que definirlos nuevamente en la clase envoltoria.


In [ ]:
    def __getattr__(self, name):
        # Check if the attribute exists in the underlying dataset
        if hasattr(self.dataset, name):
            return getattr(self.dataset, name)
        raise AttributeError(
            f"'{type(self).__name__}' and its underlying dataset have no attribute '{name}'"
        )

### Clase DataModule

Esta clase es una subclase de `pytorch_lightning.LightningDataModule` y se utiliza para definir el módulo de datos en PyTorch Lightning. Aquí está una descripción de sus componentes:

- `def __init__(self, hparams, dataset=None)`: El método de inicialización de la clase toma dos argumentos: `hparams`, que son los parámetros del hiperentrenamiento, y `dataset`, que es el conjunto de datos opcional que se puede proporcionar. 

- `super(DataModule, self).__init__()`: Llama al constructor de la clase base `LightningDataModule` para inicializar la clase.

- `self.save_hyperparameters(hparams)`: Utiliza el método `save_hyperparameters` proporcionado por PyTorch Lightning para guardar los hiperparámetros en el objeto del módulo de datos. Esto es útil para registrar y rastrear automáticamente los hiperparámetros durante el entrenamiento.

- `self._mean, self._std = None, None`: Inicializa las variables `_mean` y `_std` como `None`. Estas variables se utilizan para almacenar las medias y desviaciones estándar de los datos, si es necesario.

- `self._saved_dataloaders = dict()`: Inicializa un diccionario vacío `_saved_dataloaders` para almacenar los dataloaders generados durante el proceso de entrenamiento.

- `self.dataset = dataset`: Almacena el conjunto de datos proporcionado como atributo de la clase.

Esta clase proporciona una base para definir y configurar el módulo de datos en un proyecto de PyTorch Lightning. Al extender esta clase, se pueden implementar los métodos necesarios para preparar y cargar los datos, así como para configurar los dataloaders y realizar cualquier preprocesamiento adicional necesario.


In [ ]:
class DataModule(LightningDataModule):
    def __init__(self, hparams, dataset=None):
        super(DataModule, self).__init__()
        self.save_hyperparameters(hparams)
        self._mean, self._std = None, None
        self._saved_dataloaders = dict()
        self.dataset = dataset

### Método setup

Este método se utiliza para preparar los datos para el entrenamiento, incluyendo la carga del conjunto de datos, la creación de divisiones de entrenamiento, validación y prueba, y cualquier preprocesamiento adicional necesario. Aquí está una descripción de su funcionalidad:

- `def setup(self, stage)`: Define un método llamado `setup` que toma `stage` como argumento, indicando la etapa del proceso de entrenamiento.

- `if self.dataset is None:`: Comprueba si el conjunto de datos ya está definido. Si no lo está, procede a cargar el conjunto de datos según la configuración especificada en los hiperparámetros.

- `if self.hparams["dataset"] == "Custom":`: Si se especifica un conjunto de datos personalizado en los hiperparámetros, se crea una instancia del conjunto de datos personalizado (`datasets.Custom`) utilizando los archivos de coordenadas, incrustaciones, energías y fuerzas proporcionados en los hiperparámetros.

- `else:`: Si se utiliza un conjunto de datos predefinido, se crea una instancia del conjunto de datos correspondiente utilizando la configuración de ruta y cualquier argumento adicional proporcionado en los hiperparámetros.

- `self.dataset = FloatCastDatasetWrapper(...)`: Una vez que se carga el conjunto de datos, se envuelve en un `FloatCastDatasetWrapper` para asegurarse de que todos los tensores sean del tipo de dato flotante especificado en los hiperparámetros.

- `self.idx_train, self.idx_val, self.idx_test = make_splits(...)`: Se generan divisiones de entrenamiento, validación y prueba utilizando la función `make_splits`, que toma en cuenta el tamaño de cada conjunto, la semilla aleatoria y el directorio de registros para guardar las divisiones.

- `self.train_dataset = Subset(self.dataset, self.idx_train)`: Se crea un subconjunto de entrenamiento a partir del conjunto de datos completo utilizando los índices de entrenamiento generados previamente.

- `self.val_dataset = Subset(self.dataset, self.idx_val)`: Se crea un subconjunto de validación a partir del conjunto de datos completo utilizando los índices de validación generados previamente.

- `self.test_dataset = Subset(self.dataset, self.idx_test)`: Se crea un subconjunto de prueba a partir del conjunto de datos completo utilizando los índices de prueba generados previamente.

- `if self.hparams["standardize"]:`: Si se especifica en los hiperparámetros, se realiza una estandarización de los datos llamando al método `_standardize`.

Este método garantiza que los datos estén preparados y divididos adecuadamente para el entrenamiento del modelo, lo que facilita la configuración y ejecución del proceso de entrenamiento en PyTorch Lightning.


In [ ]:
    def setup(self, stage):
        if self.dataset is None:
            if self.hparams["dataset"] == "Custom":
                self.dataset = datasets.Custom(
                    self.hparams["coord_files"],
                    self.hparams["embed_files"],
                    self.hparams["energy_files"],
                    self.hparams["force_files"],
                )
            else:
                dataset_arg = {}
                if self.hparams["dataset_arg"] is not None:
                    dataset_arg = self.hparams["dataset_arg"]
                self.dataset = getattr(datasets, self.hparams["dataset"])(
                    self.hparams["dataset_root"], **dataset_arg
                )
        self.dataset = FloatCastDatasetWrapper(
            self.dataset, dtype_mapping[self.hparams["precision"]]
        )

        self.idx_train, self.idx_val, self.idx_test = make_splits(
            len(self.dataset),
            self.hparams["train_size"],
            self.hparams["val_size"],
            self.hparams["test_size"],
            self.hparams["seed"],
            join(self.hparams["log_dir"], "splits.npz"),
            self.hparams["splits"],
        )
        print(
            f"train {len(self.idx_train)}, val {len(self.idx_val)}, test {len(self.idx_test)}"
        )

        self.train_dataset = Subset(self.dataset, self.idx_train)
        self.val_dataset = Subset(self.dataset, self.idx_val)
        self.test_dataset = Subset(self.dataset, self.idx_test)

        if self.hparams["standardize"]:
            self._standardize()

### Método train_dataloader

Este método se utiliza para crear y devolver un DataLoader para el conjunto de datos de entrenamiento. Aquí está una descripción de su funcionalidad:

- `def train_dataloader(self)`: Define un método llamado `train_dataloader` que no toma ningún argumento.

- `return self._get_dataloader(self.train_dataset, "train")`: Devuelve un DataLoader para el conjunto de datos de entrenamiento llamando al método `_get_dataloader`. El método `_get_dataloader` se encarga de crear el DataLoader con el conjunto de datos especificado y un indicador para identificar el tipo de DataLoader, que en este caso es "train".


In [ ]:
    def train_dataloader(self):
        return self._get_dataloader(self.train_dataset, "train")

### Método val_dataloader

Este método se utiliza para crear y devolver los DataLoaders para el conjunto de datos de validación y, opcionalmente, para el conjunto de datos de prueba. Aquí está una descripción de su funcionalidad:

- `def val_dataloader(self)`: Define un método llamado `val_dataloader` que no toma ningún argumento.

- `loaders = [self._get_dataloader(self.val_dataset, "val")]`: Crea un DataLoader para el conjunto de datos de validación llamando al método `_get_dataloader`. Este DataLoader se agrega a una lista llamada `loaders`.

- `if (...)`: Comprueba si hay un conjunto de datos de prueba disponible y si es el momento de incluirlo en la validación, según el intervalo de prueba especificado en los hiperparámetros y el número actual de épocas.

- `loaders.append(self._get_dataloader(self.test_dataset, "test"))`: Si se cumple la condición anterior, se crea un DataLoader para el conjunto de datos de prueba y se agrega a la lista `loaders`.

- `return loaders`: Devuelve la lista de DataLoaders, que contiene los DataLoaders para los conjuntos de datos de validación y, opcionalmente, de prueba.

Este método proporciona una forma flexible de obtener los DataLoaders para los conjuntos de datos de validación y prueba, lo que permite realizar evaluaciones durante el entrenamiento y ajustar el proceso de entrenamiento según sea necesario.


In [ ]:
    def val_dataloader(self):
        loaders = [self._get_dataloader(self.val_dataset, "val")]
        if (
            len(self.test_dataset) > 0
            and (self.trainer.current_epoch + 1) % self.hparams["test_interval"] == 0
        ):
            loaders.append(self._get_dataloader(self.test_dataset, "test"))
        return loaders

### Método test_dataloader

Este método se utiliza para crear y devolver un DataLoader para el conjunto de datos de prueba. Aquí está una descripción de su funcionalidad:

- `def test_dataloader(self)`: Define un método llamado `test_dataloader` que no toma ningún argumento.

- `return self._get_dataloader(self.test_dataset, "test")`: Devuelve un DataLoader para el conjunto de datos de prueba llamando al método `_get_dataloader`. El método `_get_dataloader` se encarga de crear el DataLoader con el conjunto de datos especificado y un indicador para identificar el tipo de DataLoader, que en este caso es "test".


In [ ]:
    def test_dataloader(self):
        return self._get_dataloader(self.test_dataset, "test")

### Método atomref (propiedad)

Este método es una propiedad que se utiliza para obtener la referencia a los átomos del conjunto de datos. Aquí está una descripción de su funcionalidad:

- `@property`: Este decorador indica que `atomref` es una propiedad, lo que permite acceder a ella como un atributo sin necesidad de llamar a un método.

- `def atomref(self)`: Define un método llamado `atomref` que toma `self` como argumento.

- `if hasattr(self.dataset, "get_atomref"):`: Comprueba si el conjunto de datos tiene un método llamado `get_atomref`. Si el método existe, se llama a `self.dataset.get_atomref()` para obtener la referencia a los átomos.

- `return self.dataset.get_atomref()`: Devuelve la referencia a los átomos del conjunto de datos si el método `get_atomref` está disponible en el conjunto de datos.

- `return None`: Si el conjunto de datos no tiene un método `get_atomref`, devuelve `None`.

Este método proporciona una forma de obtener la referencia a los átomos del conjunto de datos, lo que puede ser útil para realizar ciertas operaciones o análisis adicionales en el conjunto de datos.


In [ ]:
    @property
    def atomref(self):
        if hasattr(self.dataset, "get_atomref"):
            return self.dataset.get_atomref()
        return None

### Propiedad mean

Esta propiedad se utiliza para acceder a la media de los datos del conjunto de datos. Aquí está una descripción de su funcionalidad:

- `@property`: Este decorador indica que `mean` es una propiedad, lo que permite acceder a ella como un atributo sin necesidad de llamar a un método.

- `def mean(self)`: Define un método llamado `mean` que toma `self` como argumento.

- `return self._mean`: Devuelve el valor almacenado en la variable `_mean`, que representa la media de los datos del conjunto de datos.

Esta propiedad proporciona una forma de acceder a la media de los datos del conjunto de datos desde fuera de la clase, lo que puede ser útil para realizar análisis o visualizaciones adicionales.


In [ ]:
    @property
    def mean(self):
        return self._mean

### Propiedad std

Esta propiedad se utiliza para acceder a la desviación estándar de los datos del conjunto de datos. Aquí está una descripción de su funcionalidad:

- `@property`: Este decorador indica que `std` es una propiedad, lo que permite acceder a ella como un atributo sin necesidad de llamar a un método.

- `def std(self)`: Define un método llamado `std` que toma `self` como argumento.

- `return self._std`: Devuelve el valor almacenado en la variable `_std`, que representa la desviación estándar de los datos del conjunto de datos.

Esta propiedad proporciona una forma de acceder a la desviación estándar de los datos del conjunto de datos desde fuera de la clase, lo que puede ser útil para realizar análisis o visualizaciones adicionales.


In [ ]:
    @property
    def std(self):
        return self._std

### Método _get_dataloader

Este método se utiliza para obtener un DataLoader para un conjunto de datos específico en una determinada etapa del proceso, como entrenamiento, validación o prueba. Aquí está una descripción de su funcionalidad:

- `def _get_dataloader(self, dataset, stage, store_dataloader=True)`: Define un método privado llamado `_get_dataloader` que toma tres argumentos: `dataset`, el conjunto de datos para el que se desea crear el DataLoader; `stage`, la etapa del proceso para la que se necesita el DataLoader (por ejemplo, "train", "val" o "test"); y `store_dataloader`, un indicador booleano que determina si se debe almacenar el DataLoader para su posterior uso.

- `store_dataloader = (...)` y `if stage in self._saved_dataloaders and store_dataloader`: Estos bloques de código gestionan si se debe almacenar el DataLoader para su posterior uso. Si `store_dataloader` es `True` y ya existe un DataLoader almacenado para la etapa específica, se devuelve el DataLoader almacenado en lugar de crear uno nuevo.

- En la sección siguiente, se configura el DataLoader según la etapa especificada:
    - Para la etapa de entrenamiento (`stage == "train"`), se utiliza el tamaño de lote y la opción de barajado especificados en los hiperparámetros.
    - Para las etapas de validación y prueba (`stage in ["val", "test"]`), se utiliza el tamaño de lote de inferencia especificado en los hiperparámetros y la opción de barajado se establece en `False`.

- `dl = DataLoader(...)`: Se crea el DataLoader con los parámetros especificados para el conjunto de datos.

- `if store_dataloader:` y `self._saved_dataloaders[stage] = dl`: Si `store_dataloader` es `True`, se almacena el DataLoader en el diccionario `_saved_dataloaders` para su posterior uso.

- `return dl`: Se devuelve el DataLoader creado o almacenado.

Este método proporciona una forma de obtener y gestionar DataLoaders de manera eficiente para diferentes etapas del proceso, garantizando que los DataLoaders se creen solo cuando sea necesario y se puedan almacenar para su posterior reutilización si es necesario.


In [ ]:
    def _get_dataloader(self, dataset, stage, store_dataloader=True):
        store_dataloader = (
            store_dataloader and self.trainer.reload_dataloaders_every_n_epochs <= 0
        )
        if stage in self._saved_dataloaders and store_dataloader:
            # storing the dataloaders like this breaks calls to trainer.reload_train_val_dataloaders
            # but makes it possible that the dataloaders are not recreated on every testing epoch
            return self._saved_dataloaders[stage]

        if stage == "train":
            batch_size = self.hparams["batch_size"]
            shuffle = True
        elif stage in ["val", "test"]:
            batch_size = self.hparams["inference_batch_size"]
            shuffle = False

        dl = DataLoader(
            dataset=dataset,
            batch_size=batch_size,
            num_workers=self.hparams["num_workers"],
            pin_memory=True,
            shuffle=shuffle,
        )

        if store_dataloader:
            self._saved_dataloaders[stage] = dl
        return dl

### Método _standardize

Este método se utiliza para estandarizar los datos del conjunto de entrenamiento, calculando la media y la desviación estándar de las energías en el conjunto de datos de entrenamiento. Aquí está una descripción de su funcionalidad:

- `def _standardize(self)`: Define un método privado llamado `_standardize` que no toma ningún argumento.

- `def get_energy(batch, atomref)`: Define una función interna llamada `get_energy` que toma un lote de datos y una referencia de átomos como argumentos y devuelve las energías del lote.

- `data = tqdm(...)`: Utiliza la función `tqdm` para iterar sobre un DataLoader que contiene el conjunto de datos de entrenamiento y calcular la media y la desviación estándar de las energías en el conjunto de datos.

- `atomref = ...`: Determina si se está utilizando un modelo de referencia de átomos. Si es así, se utiliza la referencia de átomos del módulo de datos (`self.atomref`), de lo contrario, se establece en `None`.

- `ys = torch.cat(...)`: Extrae las energías de los lotes de datos y las concatena en un tensor `ys`.

- `self._mean = ys.mean(dim=0)`: Calcula la media de las energías a lo largo de la dimensión 0 y la almacena en el atributo `_mean`.

- `self._std = ys.std(dim=0)`: Calcula la desviación estándar de las energías a lo largo de la dimensión 0 y la almacena en el atributo `_std`.

Este método es importante para normalizar los datos de entrada durante el entrenamiento del modelo, lo que puede ayudar a mejorar la convergencia y el rendimiento del modelo. La media y la desviación estándar calculadas se utilizan luego para normalizar los datos de entrada durante la inferencia.


In [ ]:
    def _standardize(self):
        def get_energy(batch, atomref):
            if "y" not in batch or batch.y is None:
                raise MissingEnergyException()

            if atomref is None:
                return batch.y.clone()

            # remove atomref energies from the target energy
            atomref_energy = scatter(atomref[batch.z], batch.batch, dim=0)
            return (batch.y.squeeze() - atomref_energy.squeeze()).clone()

        data = tqdm(
            self._get_dataloader(self.train_dataset, "val", store_dataloader=False),
            desc="computing mean and std",
        )
        try:
            # only remove atomref energies if the atomref prior is used
            atomref = self.atomref if self.hparams["prior_model"] == "Atomref" else None
            # extract energies from the data
            ys = torch.cat([get_energy(batch, atomref) for batch in data])
        except MissingEnergyException:
            rank_zero_warn(
                "Standardize is true but failed to compute dataset mean and "
                "standard deviation. Maybe the dataset only contains forces."
            )
            return

        # compute mean and standard deviation
        self._mean = ys.mean(dim=0)
        self._std = ys.std(dim=0)

## scripts/train.py

In [ ]:
import sys
import os
import argparse
import logging
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.loggers import CSVLogger, WandbLogger
from pytorch_lightning.strategies.ddp import DDPStrategy
from torchmdnet.module import LNNP
from torchmdnet import datasets, priors, models
from torchmdnet.data import DataModule
from torchmdnet.models import output_modules
from torchmdnet.models.model import create_prior_models
from torchmdnet.models.utils import rbf_class_mapping, act_class_mapping, dtype_mapping
from torchmdnet.utils import LoadFromFile, LoadFromCheckpoint, save_argparse, number
import torch

In [ ]:
def get_args():
    # fmt: off
    parser = argparse.ArgumentParser(description='Training')
    parser.add_argument('--load-model', action=LoadFromCheckpoint, help='Restart training using a model checkpoint')  # keep first
    parser.add_argument('--conf', '-c', type=open, action=LoadFromFile, help='Configuration yaml file')  # keep second
    parser.add_argument('--num-epochs', default=300, type=int, help='number of epochs')
    parser.add_argument('--batch-size', default=32, type=int, help='batch size')
    parser.add_argument('--inference-batch-size', default=None, type=int, help='Batchsize for validation and tests.')
    parser.add_argument('--lr', default=1e-4, type=float, help='learning rate')
    parser.add_argument('--lr-patience', type=int, default=10, help='Patience for lr-schedule. Patience per eval-interval of validation')
    parser.add_argument('--lr-metric', type=str, default='val_loss', choices=['train_loss', 'val_loss'], help='Metric to monitor when deciding whether to reduce learning rate')
    parser.add_argument('--lr-min', type=float, default=1e-6, help='Minimum learning rate before early stop')
    parser.add_argument('--lr-factor', type=float, default=0.8, help='Factor by which to multiply the learning rate when the metric stops improving')
    parser.add_argument('--lr-warmup-steps', type=int, default=0, help='How many steps to warm-up over. Defaults to 0 for no warm-up')
    parser.add_argument('--early-stopping-patience', type=int, default=30, help='Stop training after this many epochs without improvement')
    parser.add_argument('--reset-trainer', type=bool, default=False, help='Reset training metrics (e.g. early stopping, lr) when loading a model checkpoint')
    parser.add_argument('--weight-decay', type=float, default=0.0, help='Weight decay strength')
    parser.add_argument('--ema-alpha-y', type=float, default=1.0, help='The amount of influence of new losses on the exponential moving average of y')
    parser.add_argument('--ema-alpha-neg-dy', type=float, default=1.0, help='The amount of influence of new losses on the exponential moving average of dy')
    parser.add_argument('--ngpus', type=int, default=-1, help='Number of GPUs, -1 use all available. Use CUDA_VISIBLE_DEVICES=1, to decide gpus')
    parser.add_argument('--num-nodes', type=int, default=1, help='Number of nodes')
    parser.add_argument('--precision', type=int, default=32, choices=[16, 32, 64], help='Floating point precision')
    parser.add_argument('--log-dir', '-l', default='/tmp/logs', help='log file')
    parser.add_argument('--splits', default=None, help='Npz with splits idx_train, idx_val, idx_test')
    parser.add_argument('--train-size', type=number, default=None, help='Percentage/number of samples in training set (None to use all remaining samples)')
    parser.add_argument('--val-size', type=number, default=0.05, help='Percentage/number of samples in validation set (None to use all remaining samples)')
    parser.add_argument('--test-size', type=number, default=0.1, help='Percentage/number of samples in test set (None to use all remaining samples)')
    parser.add_argument('--test-interval', type=int, default=10, help='Test interval, one test per n epochs (default: 10)')
    parser.add_argument('--save-interval', type=int, default=10, help='Save interval, one save per n epochs (default: 10)')
    parser.add_argument('--seed', type=int, default=1, help='random seed (default: 1)')
    parser.add_argument('--num-workers', type=int, default=4, help='Number of workers for data prefetch')
    parser.add_argument('--redirect', type=bool, default=False, help='Redirect stdout and stderr to log_dir/log')
    parser.add_argument('--gradient-clipping', type=float, default=0.0, help='Gradient clipping norm')

    # dataset specific
    parser.add_argument('--dataset', default=None, type=str, choices=datasets.__all__, help='Name of the torch_geometric dataset')
    parser.add_argument('--dataset-root', default='~/data', type=str, help='Data storage directory (not used if dataset is "CG")')
    parser.add_argument('--dataset-arg', default=None, type=str, help='Additional dataset arguments, e.g. target property for QM9 or molecule for MD17. Need to be specified in JSON format i.e. \'{"molecules": "aspirin,benzene"}\'')
    parser.add_argument('--coord-files', default=None, type=str, help='Custom coordinate files glob')
    parser.add_argument('--embed-files', default=None, type=str, help='Custom embedding files glob')
    parser.add_argument('--energy-files', default=None, type=str, help='Custom energy files glob')
    parser.add_argument('--force-files', default=None, type=str, help='Custom force files glob')
    parser.add_argument('--y-weight', default=1.0, type=float, help='Weighting factor for y label in the loss function')
    parser.add_argument('--neg-dy-weight', default=1.0, type=float, help='Weighting factor for neg_dy label in the loss function')

    # model architecture
    parser.add_argument('--model', type=str, default='graph-network', choices=models.__all__, help='Which model to train')
    parser.add_argument('--output-model', type=str, default='Scalar', choices=output_modules.__all__, help='The type of output model')
    parser.add_argument('--prior-model', type=str, default=None, choices=priors.__all__, help='Which prior model to use')

    # architectural args
    parser.add_argument('--charge', type=bool, default=False, help='Model needs a total charge')
    parser.add_argument('--spin', type=bool, default=False, help='Model needs a spin state')
    parser.add_argument('--embedding-dimension', type=int, default=256, help='Embedding dimension')
    parser.add_argument('--num-layers', type=int, default=6, help='Number of interaction layers in the model')
    parser.add_argument('--num-rbf', type=int, default=64, help='Number of radial basis functions in model')
    parser.add_argument('--activation', type=str, default='silu', choices=list(act_class_mapping.keys()), help='Activation function')
    parser.add_argument('--rbf-type', type=str, default='expnorm', choices=list(rbf_class_mapping.keys()), help='Type of distance expansion')
    parser.add_argument('--trainable-rbf', type=bool, default=False, help='If distance expansion functions should be trainable')
    parser.add_argument('--neighbor-embedding', type=bool, default=False, help='If a neighbor embedding should be applied before interactions')
    parser.add_argument('--aggr', type=str, default='add', help='Aggregation operation for CFConv filter output. Must be one of \'add\', \'mean\', or \'max\'')

    # Transformer specific
    parser.add_argument('--distance-influence', type=str, default='both', choices=['keys', 'values', 'both', 'none'], help='Where distance information is included inside the attention')
    parser.add_argument('--attn-activation', default='silu', choices=list(act_class_mapping.keys()), help='Attention activation function')
    parser.add_argument('--num-heads', type=int, default=8, help='Number of attention heads')
    
    # TensorNet specific
    parser.add_argument('--equivariance-invariance-group', type=str, default='O(3)', help='Equivariance and invariance group of TensorNet')

    # other args
    parser.add_argument('--derivative', default=False, type=bool, help='If true, take the derivative of the prediction w.r.t coordinates')
    parser.add_argument('--cutoff-lower', type=float, default=0.0, help='Lower cutoff in model')
    parser.add_argument('--cutoff-upper', type=float, default=5.0, help='Upper cutoff in model')
    parser.add_argument('--atom-filter', type=int, default=-1, help='Only sum over atoms with Z > atom_filter')
    parser.add_argument('--max-z', type=int, default=100, help='Maximum atomic number that fits in the embedding matrix')
    parser.add_argument('--max-num-neighbors', type=int, default=32, help='Maximum number of neighbors to consider in the network')
    parser.add_argument('--standardize', type=bool, default=False, help='If true, multiply prediction by dataset std and add mean')
    parser.add_argument('--reduce-op', type=str, default='add', choices=['add', 'mean'], help='Reduce operation to apply to atomic predictions')
    parser.add_argument('--wandb-use', default=False, type=bool, help='Defines if wandb is used or not')
    parser.add_argument('--wandb-name', default='training', type=str, help='Give a name to your wandb run')
    parser.add_argument('--wandb-project', default='training_', type=str, help='Define what wandb Project to log to')
    parser.add_argument('--wandb-resume-from-id', default=None, type=str, help='Resume a wandb run from a given run id. The id can be retrieved from the wandb dashboard')
    parser.add_argument('--tensorboard-use', default=False, type=bool, help='Defines if tensor board is used or not')

    # fmt: on

    args = parser.parse_args()

    if args.redirect:
        sys.stdout = open(os.path.join(args.log_dir, "log"), "w")
        sys.stderr = sys.stdout
        logging.getLogger("pytorch_lightning").addHandler(
            logging.StreamHandler(sys.stdout)
        )

    if args.inference_batch_size is None:
        args.inference_batch_size = args.batch_size

    save_argparse(args, os.path.join(args.log_dir, "input.yaml"), exclude=["conf"])

    return args

In [ ]:
def main():
    args = get_args()
    pl.seed_everything(args.seed, workers=True)

    # initialize data module
    data = DataModule(args)
    data.prepare_data()
    data.setup("fit")

    prior_models = create_prior_models(vars(args), data.dataset)
    args.prior_args = [p.get_init_args() for p in prior_models]

    # initialize lightning module
    model = LNNP(args, prior_model=prior_models, mean=data.mean, std=data.std)

    checkpoint_callback = ModelCheckpoint(
        dirpath=args.log_dir,
        monitor="val_loss",
        save_top_k=10,  # -1 to save all
        every_n_epochs=args.save_interval,
        filename="{epoch}-{val_loss:.4f}-{test_loss:.4f}",
    )
    early_stopping = EarlyStopping("val_loss", patience=args.early_stopping_patience)

    csv_logger = CSVLogger(args.log_dir, name="", version="")
    _logger = [csv_logger]
    if args.wandb_use:
        wandb_logger = WandbLogger(
            project=args.wandb_project,
            name=args.wandb_name,
            save_dir=args.log_dir,
            resume="must" if args.wandb_resume_from_id is not None else None,
            id=args.wandb_resume_from_id,
        )
        _logger.append(wandb_logger)

    if args.tensorboard_use:
        tb_logger = pl.loggers.TensorBoardLogger(
            args.log_dir, name="tensorbord", version="", default_hp_metric=False
        )
        _logger.append(tb_logger)

    trainer = pl.Trainer(
        strategy=DDPStrategy(find_unused_parameters=False),
        max_epochs=args.num_epochs,
        gpus=args.ngpus,
        num_nodes=args.num_nodes,
        default_root_dir=args.log_dir,
        auto_lr_find=False,
        resume_from_checkpoint=None if args.reset_trainer else args.load_model,
        callbacks=[early_stopping, checkpoint_callback],
        logger=_logger,
        precision=args.precision,
        gradient_clip_val=args.gradient_clipping,
    )

    trainer.fit(model, data)

    # run test set after completing the fit
    model = LNNP.load_from_checkpoint(trainer.checkpoint_callback.best_model_path)
    trainer = pl.Trainer(logger=_logger)
    trainer.test(model, data)

In [ ]:
if __name__ == "__main__":
    main()